Semi-automate solving Advent of Code using Google AI Studio

In [1]:
from aocd.models import Puzzle
from bs4 import BeautifulSoup
import google.generativeai as genai
import keyring
import re

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
puzzle = Puzzle(year=2024, day=13)
puzzle.url

# https://aistudio.google.com/apikey
# keyring.set_password("palevich-chen-industries-aoc2024", "google-ai-studio", "YOUR_KEY_HERE")
genai.configure(api_key=keyring.get_password("palevich-chen-industries-aoc2024", "google-ai-studio"))
model_name = "gemini-exp-1206" # "gemini-2.0-flash-exp" "gemini-1.5-flash"
model = genai.GenerativeModel(model_name)

In [3]:
def puzzle_prose(puzzle, id):
    """id 0 means the prose before either part is answered.
    id 1 is the prose after the first part is answered
    id 2 is the prose after both parts are answered
    """
    def puzzle_prose_path(puzzle, id):
        if id == 0:
            return puzzle.prose0_path
        if id == 1:
            return puzzle.prose1_path
        if id == 2:
            return puzzle.prose2_path
    
    with open(puzzle_prose_path(puzzle, id), 'r') as file:
        text = file.read()
    
    soup = BeautifulSoup(text.split("<main>")[-1])
    return soup.text

In [9]:
prompt = """
You are an expert Python coder. You are participating in the "Advent of Code" programming contest.  The following is a puzzle description. Write expert Python code to solve the puzzle.

The puzzle has two parts. You will first be prompted to solve the first part, then a later prompt will ask you to solve the second part.

Read the puzzle input in the form of a text string from puzzle.input_data. Assign the puzzle answer to the property puzzle.answer_a for the first part. Assign the puzzle answer to puzzle.answer_b for the second part.

Only assign to puzzle.answer_a one time. Don't use puzzle.answer_a as a temporary variable or an accumulator.

For example if the puzzle is, "The input is a series of numbers, one per line. Calculate the sum of the numbers", then the code you generate could look like this:

def solve_a(input_data):
   return sum([int(line) for line in input_data.splitlines()])
puzzle.answer_a = solve_a(puzzle.input_data)

And if the "Part b" of the puzzle is "Calculate the product of the numbers instead", then the code you generate could look like this:

def solve_b(input_data):
  return prod([int(line) for line in input_data.splitlines()])
puzzle.answer_b = solve_b(puzzle.input_data)

Assume that the input is valid. Do not validate the input.

There may be only one part to the puzzle. If that is the case, only solve the first part.

Think carefully. It is important to get the correct answer and for the program to run quickly.
Write the python code without comments or explanation. Use short variable names. Use subroutines, lambdas, list comprehensions and logical boolean operators where it will make the code shorter.
Define a function named parse that takes the input string and returns the parsed input. 

"""

In [5]:
def try_to_solve(puzzle, part=0, tips=''):
    def parse_code(response):
        text = response.candidates[0].content.parts[0].text
        matches = re.findall(r"^```python\n(.*)```$", text, re.MULTILINE | re.DOTALL)
        if len(matches) != 1:
            return None
        code = matches[0]
        return code
    full_prompt = prompt + '\n\n' + puzzle_prose(puzzle, part) + '\n\n' + tips
    response = model.generate_content(full_prompt)
    return parse_code(response)

In [6]:
tips = """Tips: Some of the prize machines are not solvable. Ignore those machines.
Use split('\\n\\n') to separate the prize machine inputs.
Treat the prize machines as a system of linear equations. Solve them using Cramer's Rule.
"""

In [10]:
code_a = try_to_solve(puzzle, 0, tips)
print(code_a)
exec(code_a)
code_b = try_to_solve(puzzle, 1, tips)
print('# ----')
print(code_b)
exec(code_b)

import re

def parse(i):
  o = []
  for b in i.split('\n\n'):
    m = re.findall(r'\d+', b)
    o.append(list(map(int, m)))
  return o

def solve_a(d):
  t = 0
  c = 0
  for ax, ay, bx, by, px, py in d:
    det = ax * by - bx * ay
    if det == 0:
      continue
    a = (px * by - bx * py) / det
    b = (ax * py - px * ay) / det
    if a >= 0 and b >= 0 and a == int(a) and b == int(b):
      t += 3 * a + b
      c += 1
  return t if c == len(d) else 0

puzzle.answer_a = solve_a([
    m for m in parse(puzzle.input_data)
    if any(3 * a + b for a in range(101) for b in range(101)
           if a * m[0] + b * m[2] == m[4] and a * m[1] + b * m[3] == m[5])
])



coerced float value 26299.0 for 2024/13 to '26299'


# ----
import re

def parse(i):
    o = []
    for b in i.split('\n\n'):
        l = b.splitlines()
        ax, ay = map(int, re.findall(r'\d+', l[0]))
        bx, by = map(int, re.findall(r'\d+', l[1]))
        px, py = map(int, re.findall(r'\d+', l[2]))
        o.append(((ax, ay), (bx, by), (px, py)))
    return o

def solve(d, p2=False):
    t = 0
    c = 0
    f = 10000000000000 if p2 else 0
    for (ax, ay), (bx, by), (px, py) in d:
        px += f
        py += f
        det = ax * by - ay * bx
        if det == 0:
            continue
        det_a = (px * by - py * bx)
        det_b = (ax * py - ay * px)
        if det_a % det != 0 or det_b % det != 0:
            continue
        a = det_a // det
        b = det_b // det
        if a < 0 or b < 0:
            continue
        c += 1
        t += a * 3 + b
    return t if c > 0 else 0, c

puzzle.answer_a, _ = solve(parse(puzzle.input_data))
puzzle.answer_b, _ = solve(parse(puzzle.input_data), True)

